In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
zip_path = "drive/MyDrive/Andmeteaduse magister/4 Semester/Network Science/dataset.zip" # <Path to dataset zip>
!unzip "{zip_path}"

Archive:  drive/MyDrive/Andmeteaduse magister/4 Semester/Network Science/dataset.zip
  inflating: edges_en.csv            
  inflating: edges_et.csv            
  inflating: nodes_en.csv            
  inflating: nodes_et.csv            


In [73]:
nodes_et = pd.read_csv("nodes_et.csv")
edges_et = pd.read_csv("edges_et.csv")

# Ensure no overlap with en nodes before merging dfs
nodes_et["node_id"] = nodes_et["node_id"] + 100_000
edges_et["source"] = edges_et["source"] + 100_000
edges_et["target"] = edges_et["target"] + 100_000

nodes_en = pd.read_csv("nodes_en.csv")
edges_en = pd.read_csv("edges_en.csv")

In [74]:
nodes_en = nodes_en[~nodes_en["node_summary"].isna()]
nodes_et = nodes_et[~nodes_et["node_summary"].isna()]

In [75]:
nodes_en["lang"] = "en"
nodes_et["lang"] = "et"

In [78]:
all_nodes = pd.concat([nodes_en, nodes_et])
all_nodes = all_nodes.rename(columns={"node_et_equivalent": "interlanguagelink"}).reset_index(drop=True)

interlanguagelinks = nodes_et.merge(nodes_en, left_on="node_name", right_on="node_et_equivalent")[["node_id_x", "node_id_y"]]
interlanguagelinks.columns = ["source", "target"]
interlanguagelinks
all_edges = pd.concat([interlanguagelinks, edges_et, edges_en])

In [46]:
!pip install laserembeddings[ja]
!python -m laserembeddings download-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


# Calculate text embeddings

In [61]:
from laserembeddings import Laser
laser = Laser()

In [63]:
all_embeddings = laser.embed_sentences(all_nodes.node_summary.values, lang=all_nodes.lang.values)

In [81]:
all_embedding_df = pd.DataFrame(all_embeddings)
all_embedding_df.columns = [f"emb_{i}" for i in range(1, 1025)]

In [82]:
nodes_with_embeddings = pd.concat([all_nodes, all_embedding_df], axis=1)

In [84]:
nodes_with_embeddings.to_csv("all_nodes_with_embeddings.csv")

In [85]:
all_edges.to_csv("all_edges_interlanguage.csv")